In [86]:
import pandas as pd
import os
import nibabel as nib

def check_nii_meta():
    nii_path = os.path.abspath('../data/keyword_uniformity_test/abilities_uniformity-test_z_FDR_0.01.nii.gz')
    nii_file = nib.load(nii_path)
    print(nii_file)

In [158]:
# 하나의 nii 파일에 담겨있는 메타 데이터 확인
check_nii_meta()

In [88]:
def check_nii_activation():
    nii_path = os.path.abspath('../data/keyword_uniformity_test/abilities_uniformity-test_z_FDR_0.01.nii.gz')
    nii_file = nib.load(nii_path)
    nii_data = nii_file.get_data()
    print('nii activation data format :', nii_data.shape)
    print('first 3 rows of activation score data')
    for i, row in enumerate(nii_data):
        if i > 2:|
            break
        print(row)

In [159]:
# nii 파일 하나가 포함하는 activation score 데이터 확인
# 3차원 배열 형태이며, 각 value는 특정 인덱스의 복셀이 활성화 된 정도를 나타낸다.
check_nii_activation()

In [3]:
def check_only_activated():
    keyword = 'abilities'
    nii_path = os.path.abspath('../data/keyword_uniformity_test/%s_uniformity-test_z_FDR_0.01.nii.gz' % keyword)
    nii_file = nib.load(nii_path)
    nii_data = nii_file.get_data()
    
    first_activated = False
    activated_row = []
    for i in range(len(nii_data)):
        for j in range(len(nii_data[0])):
            for k in range(len(nii_data[0][0])):
                if abs(nii_data[i][j][k]) != 0.0:
                    cur_id = '%d_%d_%d' % (i, j, k)
                    cur_score = nii_data[i][j][k]
                    activated_row.append([cur_id, keyword, cur_score])
                    
    nii_df = pd.DataFrame(data=activated_row)
    nii_df.columns = ['voxel_id', 'keyword', 'score']
    print('keyword: %s' % keyword)
    print('total actived voxels: %d\n' % nii_df.shape[0])
    print(nii_df.head())

In [5]:
# 실제 활성화 된 복셀 인덱스를 찾아서 정리한 모습
# 1335개의 nii 파일들을 모두 불러와, 이 처럼 활성화 된 데이터만 찾아서 csv 파일에 쓴다.
# 그 결과로 전체 nii 파일에서 활성화 정보만 담고 있는 압축 텍스트 파일이 만들어진다.
check_only_activated()

FileNotFoundError: No such file or no access: '/Users/junny/brain_factorization/data/keyword_uniformity_test/abilities_uniformity-test_z_FDR_0.01.nii.gz'

In [6]:
def check_sorted_activated():
    sorted_path = os.path.abspath('../data/results/sorted_sample.csv')
    sorted_df = pd.read_csv(sorted_path)
    print(sorted_df.head())

In [7]:
# 압축 텍스트 파일은 다시 복셀 아이디로 sorting을 한다. 
# 그 결과로 동일한 복셀 아이디 별로 어떤 키워드에 얼마나 활성화 되었는 지를 확인할 수 있다.
# 이를 기반으로 X축은 복셀 아이디, Y 축은 키워드, value는 score인 sparse matrix를 구성할 수 있다.
check_sorted_activated()

FileNotFoundError: File b'/Users/junny/brain_factorization/data/results/sorted_sample.csv' does not exist

In [143]:
from scipy.sparse import csr_matrix
import csv

# sparse matrix를 만들어주는 함수
def get_csr_matrix(filepath):
    indptr = [0]
    indices = []
    data = []
    vocabulary = {}
    voxel_id_set = set()
    total_rows = get_num_lines(filepath)
    file_csv = csv.reader(open(filepath, 'r'))
    prev_voxel_id = None
    
    count = 0
    for i, row in enumerate(file_csv):
        count += 1
        if i == 0:
            continue
        if not prev_voxel_id:
            prev_voxel_id = row[0]
        voxel_id_set.add(row[0])
        index = vocabulary.setdefault(row[1], len(vocabulary))
        indices.append(index)
        data.append(float(row[2]))

        if row[0] != prev_voxel_id:
            indptr.append(i - 1)
            prev_voxel_id = row[0]
    indptr.append(count - 1)
    voxel_id_list = list(voxel_id_set)
    voxel_id_list.sort()
    nii_csr_matrix = csr_matrix((data, indices, indptr), dtype=int)
    return voxel_id_list, nii_csr_matrix, vocabulary

In [156]:
# 활성화 영역 정보만 압축한 텍스트 가운데 일부에 해당하는 샘플 데이터를 읽어옴
# 이를 기반으로 Matrix Factorization을 진행하기 위한 Sparse Data 형식을 만듬
def check_sparse_matrix():
    matrix_sample_path = os.path.abspath('../data/results/matrix_sample.csv')
    voxel_id_list, nii_csr_matrix, vocabulary = get_csr_matrix(matrix_sample_path)
    matrix_df = pd.DataFrame(data=nii_csr_matrix.toarray(), index = voxel_id_list)
    matrix_df.columns = vocabulary.keys()
    pd.options.display.max_columns = 10
    print(matrix_df.shape)
    print(matrix_df)

In [162]:
check_sparse_matrix()